# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv file exported from part I
pt1_load_file = '../exported_weather_files.csv'

#create dataframe out of csv from part I
pt2_city_info = pd.read_csv(pt1_load_file)

#drop NAs (if any) because I don't like them
pt2_city_info = pt2_city_info.dropna()

#len(pt2_city_info)

#call head on dataframe
pt2_city_info.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Cloudiness %,Humidity %,Max temp (F),Wind speed (mph)
0,0,Hilo,US,1659222514,19.7297,-155.0900,100,77,90.28,10.36
1,1,Ribeira Grande,PT,1659222810,38.5167,-28.7000,15,68,74.46,2.04
2,2,Haines Junction,CA,1659222810,60.7522,-137.5108,82,41,71.51,7.14
3,3,Lufilufi,WS,1659222811,-13.8745,-171.5986,24,79,79.79,20.11
4,4,Hong Kong,HK,1659222550,22.2855,114.1577,39,86,86.38,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
#put Lat and Lng into locations, and make it a float so it can be used in the heatmap
locations = pt2_city_info[['Lat', 'Lng']].astype(float)

#put humidity into humidity_num and also convert to float for use in heatmap
humidity_num = pt2_city_info['Humidity %'].astype(float)

In [5]:
#create a humidity heatmap layer for map
    #initialize the figure as per the docs which I will add here for future ref (https://jupyter-gmaps.readthedocs.io/en/latest/api.html)
fig=gmaps.figure()

    #create the heat layer with locations and humidity_num as weight. I think the example has dissipating as false?
layer_of_heat = gmaps.heatmap_layer(locations, weights=humidity_num, dissipating=False, gradient='red')

    #add the heat layer to the figure
fig.add_layer(layer_of_heat)
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#filter cities down to only ideal conditions mentioned below.
    #max temp greater than 70 but less than 80
    #wind speed less than 10 mph
    #zero cloudiness
idealWeather_df = pt2_city_info

#find cities with max temp greater than 70 but less than 80
idealWeather_df = idealWeather_df.loc[(idealWeather_df['Max temp (F)'] <80) & (idealWeather_df['Max temp (F)'] > 70)]

#find cities with wind speed of less than 10 mph
idealWeather_df = idealWeather_df.loc[idealWeather_df['Wind speed (mph)'] < 10]

#find cities with zero cloudiness
idealWeather_df = idealWeather_df.loc[idealWeather_df['Cloudiness %'] == 0]

#len(idealWeather_df)
    #holy smokes only 10 cities meet these criteria
idealWeather_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Cloudiness %,Humidity %,Max temp (F),Wind speed (mph)
115,115,Changji,CN,1659222850,44.0167,87.3167,0,47,77.79,4.47
269,269,Maşyāf,SY,1659222909,35.0653,36.3406,0,65,71.78,6.71
282,282,Mahon,ES,1659222913,39.8885,4.2658,0,73,75.76,4.61
298,298,Mitsamiouli,KM,1659222920,-11.3847,43.2844,0,76,75.22,6.40
326,326,Bowling Green,US,1659222755,36.9903,-86.4436,0,69,79.54,6.91
389,389,Yabrūd,SY,1659222954,33.9674,36.6574,0,35,72.79,2.95
447,447,Lygourio,GR,1659222983,37.6167,23.0333,0,55,76.19,3.69
468,468,Petrolina,BR,1659222697,-9.3986,-40.5008,0,47,78.85,5.75
529,529,Saint-Pierre,RE,1659223014,-21.3393,55.4781,0,72,71.19,6.91
535,535,Lagoa,PT,1659223016,39.0500,-27.9833,0,68,73.31,2.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#create new dataframe to store hotel info in
hotel_df = idealWeather_df

#create parameters, as specified, for the hotel api call
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

#create a loop to pull closest hotels to the cities with ideal weather
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #make API pull
    print(f"Pulling results for {index}: {row['City']}." )
    response = requests.get(base_url, params=params).json()
    
    #call the results
    results = response['results']
    
    try:
        print(f"The Closest hotel is {results[0]['name']}")
        hotel_df.loc[index, 'Hotel name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing info, skipping...")

Pulling results for 115: Changji.
The Closest hotel is Lvye Holiday Resort
Pulling results for 269: Maşyāf.
The Closest hotel is شاطئ مايو
Pulling results for 282: Mahon.
The Closest hotel is ARTIEM Capri
Pulling results for 298: Mitsamiouli.
The Closest hotel is Foyer ADM
Pulling results for 326: Bowling Green.
The Closest hotel is Hilton Garden Inn Bowling Green
Pulling results for 389: Yabrūd.
The Closest hotel is صالة افراح الاندلس
Pulling results for 447: Lygourio.
The Closest hotel is Hani Inn
Pulling results for 468: Petrolina.
The Closest hotel is JB Hotel
Pulling results for 529: Saint-Pierre.
The Closest hotel is Alize Plage
Pulling results for 535: Lagoa.
The Closest hotel is Casa Das Faias


In [8]:
hotel_df

,Unnamed: 0,City,Country,Date,Lat,Lng,Cloudiness %,Humidity %,Max temp (F),Wind speed (mph),Hotel name
115,115,Changji,CN,1659222850,44.0167,87.3167,0,47,77.79,4.47,Lvye Holiday Resort
269,269,Maşyāf,SY,1659222909,35.0653,36.3406,0,65,71.78,6.71,شاطئ مايو
282,282,Mahon,ES,1659222913,39.8885,4.2658,0,73,75.76,4.61,ARTIEM Capri
298,298,Mitsamiouli,KM,1659222920,-11.3847,43.2844,0,76,75.22,6.40,Foyer ADM
326,326,Bowling Green,US,1659222755,36.9903,-86.4436,0,69,79.54,6.91,Hilton Garden Inn Bowling Green
389,389,Yabrūd,SY,1659222954,33.9674,36.6574,0,35,72.79,2.95,صالة افراح الاندلس
447,447,Lygourio,GR,1659222983,37.6167,23.0333,0,55,76.19,3.69,Hani Inn
468,468,Petrolina,BR,1659222697,-9.3986,-40.5008,0,47,78.85,5.75,JB Hotel
529,529,Saint-Pierre,RE,1659223014,-21.3393,55.4781,0,72,71.19,6.91,Alize Plage
535,535,Lagoa,PT,1659223016,39.0500,-27.9833,0,68,73.31,2.80,Casa Das Faias


In [9]:
#reorganize the columns so the hotel name comes first, because it looks better to me that way
hotel_df[['Hotel name', 'City', 'Country', 'Lat', 'Lng', 'Cloudiness %', 'Humidity %', 'Max temp (F)', 'Wind speed (mph)', 'Date']]

,Hotel name,City,Country,Lat,Lng,Cloudiness %,Humidity %,Max temp (F),Wind speed (mph),Date
115,Lvye Holiday Resort,Changji,CN,44.0167,87.3167,0,47,77.79,4.47,1659222850
269,شاطئ مايو,Maşyāf,SY,35.0653,36.3406,0,65,71.78,6.71,1659222909
282,ARTIEM Capri,Mahon,ES,39.8885,4.2658,0,73,75.76,4.61,1659222913
298,Foyer ADM,Mitsamiouli,KM,-11.3847,43.2844,0,76,75.22,6.40,1659222920
326,Hilton Garden Inn Bowling Green,Bowling Green,US,36.9903,-86.4436,0,69,79.54,6.91,1659222755
389,صالة افراح الاندلس,Yabrūd,SY,33.9674,36.6574,0,35,72.79,2.95,1659222954
447,Hani Inn,Lygourio,GR,37.6167,23.0333,0,55,76.19,3.69,1659222983
468,JB Hotel,Petrolina,BR,-9.3986,-40.5008,0,47,78.85,5.75,1659222697
529,Alize Plage,Saint-Pierre,RE,-21.3393,55.4781,0,72,71.19,6.91,1659223014
535,Casa Das Faias,Lagoa,PT,39.0500,-27.9833,0,68,73.31,2.80,1659223016


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Display figure
fig.add_layer(markers)
fig

print(fig)

Figure(layout=FigureLayout(height='420px'))
